In [1]:
!python --version

Python 3.7.6


In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import numpy as np
import keras
import cv2
import matplotlib.pyplot as plt
import sys,gc
import pickle
import pandas as pd
import seaborn as sns

C:\Users\sajan\AppData\Local\Programs\Python\Python37\Lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sajan\AppData\Local\Programs\Python\Python37\Lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\sajan\AppData\Local\Programs\Python\Python37\Lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\sajan\AppD

In [2]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [85]:
train = pd.read_csv('train.csv')
train_labels = train.pop('base_score')
train.drop('patient_id',axis=1, inplace=True)
train.head()

,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37
4,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43


In [86]:
test = pd.read_csv('test.csv')
test_indices = test.pop('patient_id')
test = test[train.columns]
test.head()

,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed
0,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10,28-Feb-12,22
1,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9,5-Mar-17,35
2,Zyclara,Keratosis,"""4 days in on first 2 weeks. Using on arms an...",4,3-Jul-14,13
3,Methadone,Opiate Withdrawal,"""Ive been on Methadone for over ten years and ...",7,18-Oct-16,21
4,Ambien,Insomnia,"""Ditto on rebound sleepless when discontinued....",2,13-Jan-15,44


In [87]:
train_test = pd.concat([train,test])
train_test.shape

(42925, 6)

In [88]:
use_case_dict = dict(zip(train_test.use_case_for_drug.value_counts().index,train_test.use_case_for_drug.value_counts()))
train_test.use_case_for_drug = [i if use_case_dict[i]>45 else 'other' for i in list(train_test.use_case_for_drug) ]

In [89]:
drug_name_dict = dict(zip(train_test.name_of_drug.value_counts().index,train_test.name_of_drug.value_counts()))
train_test.name_of_drug = [i if drug_name_dict[i]>np.mean(train_test.name_of_drug.value_counts())-20 else 'other' for i in list(train_test.name_of_drug) ]

In [90]:
train_test.drug_approved_by_UIC = pd.to_datetime(train_test.drug_approved_by_UIC)

In [91]:
train_test['day'] = train_test.drug_approved_by_UIC.dt.day
train_test['month'] = train_test.drug_approved_by_UIC.dt.month
train_test['year'] = train_test.drug_approved_by_UIC.dt.year
# train_test['hour'] = train_test.date_time.dt.hour
# train_test['minute'] = train_test.date_time.dt.minute
# train_test['second'] = train_test.date_time.dt.second
train_test.drop('drug_approved_by_UIC', axis=1, inplace=True)

In [92]:
train_test.drop('review_by_patient', axis=1, inplace=True)

In [93]:
train_test = pd.concat([train_test, pd.get_dummies(train_test.name_of_drug), pd.get_dummies(train_test.use_case_for_drug)], axis=1)

In [94]:
train_test.drop(['name_of_drug','use_case_for_drug'],axis=1,inplace=True)

In [95]:
stdscaler1 = StandardScaler()
stdscaler2 = StandardScaler()
# stdscaler1.fit_transform(train_test.number_of_times_prescribed).reshape(-1,1)
train_test.number_of_times_prescribed = stdscaler1.fit_transform(np.array(train_test.number_of_times_prescribed).reshape(-1,1))
train_test.effectiveness_rating = stdscaler2.fit_transform(np.array(train_test.effectiveness_rating).reshape(-1,1))

In [96]:
train_test.head()

,effectiveness_rating,number_of_times_prescribed,day,month,year,Abacavir / dolutegravir / lamivudine,Abatacept,Abilify,Abilify Discmelt,Abiraterone,...,Ulcerative Colitis,Underactive Thyroid,Urinary Incontinence,Urinary Tract Infection,Urticaria,Vaginal Yeast Infection,Weight Loss,ibromyalgia,mance Anxiety,other
0,0.613719,-0.023138,20,5,2012,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0.308371,4.640073,27,4,2010,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.607672,-0.305757,14,12,2009,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.613719,0.259481,27,11,2016,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-1.523715,0.429052,28,11,2015,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
train_test = train_test.to_numpy()

In [98]:
train_test.shape

(42925, 2502)

In [39]:
# train_test = train_test[:,2:]

In [99]:
gbt = GradientBoostingRegressor(random_state=5,verbose=1,max_depth=9, n_estimators = 1200,  criterion='friedman_mse',warm_start=True,tol=0.00001,learning_rate=0.05,validation_fraction=0.02, n_iter_no_change=100)
gbt.fit(train_test[:train.shape[0]], train_labels)

      Iter       Train Loss   Remaining Time 
         1           2.3418          205.67m
         2           2.1726          186.57m
         3           2.0199          178.79m
         4           1.8822          173.18m
         5           1.7580          169.05m
         6           1.6458          166.29m
         7           1.5441          164.48m
         8           1.4525          163.43m
         9           1.3699          163.89m
        10           1.2950          163.64m
        20           0.8339          157.71m
        30           0.5875          163.57m
        40           0.4501          172.87m
        50           0.3443          174.20m
        60           0.2795          171.10m
        70           0.2144          172.13m
        80           0.1764          168.76m
        90           0.1358          169.89m
       100           0.1013          170.97m
       200           0.0151          170.12m
       300           0.0039          160.12m
       40

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='ls', max_depth=9,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=1200,
                          n_iter_no_change=100, presort='deprecated',
                          random_state=5, subsample=1.0, tol=1e-05,
                          validation_fraction=0.02, verbose=1, warm_start=True)

In [100]:
gbt_preds = gbt.predict(train_test[train.shape[0]:])  

In [101]:
sub = pd.DataFrame()

In [102]:
sub['patient_id'] = test_indices

In [103]:
sub['base_score'] = gbt_preds#np.array(gbt_preds).astype('int')#np.mean([preds,gbt_preds]).

In [104]:
sub.to_csv('sub_gbt5.csv',index=False)

In [82]:
# GradientBoostingRegressor()

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [14]:
from catboost import Pool, CatBoostRegressor

In [15]:
train_pool = Pool(train_test[:int(train.shape[0]*0.8)], train_labels[:int(len(train_labels)*0.8)], cat_features = [0,1,4,5,6])
eval_pool = Pool(train_test[int(train.shape[0]*0.8):train.shape[0]], train_labels[int(len(train_labels)*0.8):], cat_features = [0,1,4,5,6])
test_pool = Pool(train_test[train.shape[0]:], cat_features = [0,1,4,5,6])

In [16]:
cbt = CatBoostRegressor(iterations=6000, depth=16, learning_rate=0.03, loss_function='RMSE',od_type='Iter',od_wait=100)
grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10, 12, 15, 20 ],
        'l2_leaf_reg': [1, 3, 5, 7, 9,11,13]}
# cbt.randomized_search(grid,X=train_test[:train.shape[0]],y=train_labels,plot=True)
cbt.fit(train_pool, eval_set=eval_pool,verbose=50,use_best_model=True,save_snapshot=True,snapshot_file='cbt_snapshot4.cbsnapshot')

Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.1314830637
bestIteration = 4340

Shrink model to first 4341 iterations.


In [17]:
cbt_preds = cbt.predict(test_pool)

In [74]:
import pickle
with open('gbt2.pickle','wb') as f:
	pickle.dump(gbt,f)

In [47]:
gc.collect()

22

In [83]:
l = []
for i in list(os.listdir()):
	if i.endswith('.csv') and i.startswith('sub') and not i.endswith('mean.csv'):
		df = pd.read_csv(i)
		l.append(df)
k=[]
for i in l:
	k.append(list(i.base_score))
bs = np.mean(k,axis=0)
print(bs)
sub = pd.DataFrame()
sub['patient_id'] = test_indices
sub['base_score'] = bs
sub.to_csv('sub_mean.csv',index=False)

[8.75200198 8.74013717 5.95435038 ... 6.79893875 5.30260808 8.19682391]
